In [3]:
import numpy as np
from torch import nn, optim
import torch
import random
import time

🤔思考：如何加载数据？

In [4]:
with open("labels.txt", "r") as f:
    raw = f.readlines()

tags = []
data = []
for l in raw:
    tags.append(int(l[0]))#每行的第一个字符是标签
    d = l[1:-1]#去掉标签和换行符
    d = map(float, tuple(d)) #将字符串转换为tuple，数字转换为float，方便后续转为tensor
    #tuple相对于list更省内存，因为tuple是不可变的，对象所含method更少
    d = np.array(tuple(d)).reshape(24, 16).astype(np.float32) #将tuple转换为24x16的numpy数组，并转为float32类型
    data.append(d)

#将标签和数据转为tensor，方便后续切分训练集和测试集
data = torch.tensor(data)
tags = torch.tensor(tags)

#划分训练集和测试集
train_test_ratio = 0.8
train_size = int(train_test_ratio * len(data))
test_size = len(data) - train_size
data_train = data[:train_size]
data_test  = data[train_size:]
tags_train = tags[:train_size]
tags_test  = tags[train_size:]

C:\Users\potatoshred\AppData\Local\Temp\ipykernel_25624\2786306438.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:279.)
  data = torch.tensor(data)


In [5]:
# 直接套用d2l网站上的代码，没有改动
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


batch_size = 32

w = torch.normal(0, 0.01, size=(len(data[0]), 1), requires_grad=True, dtype=torch.float32) #对每个像素都有一个权重
b = torch.zeros(1, requires_grad=True)

👇方案4：Convolutional Neural Network (CNN)

数学原理：

卷积神经网络（Convolutional Neural Network，CNN）是一种深度学习技术，它是由卷积层和池化层组成的。

- 卷积定义：

卷积，或译为叠积、褶积或旋积，是透过两个函数$f(x)$和$g(x)$的乘积，得到第三个函数$h(x)$的一种运算。
公式：

$h(x) = (f*g)(x) = \int_{-\infty}^{\infty} f(t)g(x-t)dt$


![image.png](./img/conv1.gif)


- 卷积层：

卷积层由多个卷积核组成，每个卷积核与输入图像的局部区域进行卷积操作，提取图像的特征。

![image.png](./img/conv2.gif)

卷积核可以看作一种散列函数，它对输入图像的局部区域进行加权求和，将多维的数组映射到一维。然而，与传统的散列函数不同，滑动窗口在进行加权求和时考虑了输入区域的邻近关系，从而保留了一定的局部连续性。

- 池化/汇聚层：池化层的作用是降低卷积层对图像的缩放程度，从而提高模型的鲁棒性。
  * 最大池化：最大池化是指对卷积核覆盖的区域取最大值作为输出。意义是减少特征图的尺寸并提取主要特征，减少了参数量。
  + 平均池化：平均池化是指对卷积核覆盖的区域取平均值作为输出。与最大池化相比，平均池化可以更平滑地缩减特征图的尺寸，而不太关注最显著的特征。更加关注图像中的整体分布和平均值，而不太关注局部细节。适用于提取图像分类任务中的一些全局信息。

- 全连接层：

全连接层的作用是将卷积层提取的特征连接到输出层，输出层的每个节点对应于输入图像的某个特征。




In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
# 定义CNN模型， 直接使用pytorch的API
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 卷积层1，填充1， 宽度增加2，卷积核大小3x3，输出长宽仍为16*24
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1,padding=1) 
        self.relu = nn.ReLU()  # 激活函数
        # 池化层1， 长宽均缩减一半， 输出长宽为8*12
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)  
        # # 卷积层2，卷积核大小是3x3，输出长宽为6*10
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1) 
        # 2次卷积+池化后，当前图片的大小是3x5， 64个通道。
        self.fc1 = nn.Linear(32*3*5, 128)  # 全连接层1, 输出128个特征
        self.fc2 = nn.Linear(128, 10)  # 全连接层2, 输出10个分类

    def forward(self, x):
        # 卷积层1，填充1， 宽度增加2，卷积核大小3x3，输出长宽仍为16*24
        x = self.conv1(x)
        x = self.relu(x)
        # 池化层1， 长宽均缩减一半， 输出长宽为8*12
        x = self.maxpool(x)
        # 卷积层2，卷积核大小是3x3，输出长宽为6*10
        x = self.conv2(x)
        x = self.relu(x)
        # 池化层2， 长宽均缩减一半， 输出长宽为3*5
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        # 全连接层1, 输出128个特征
        x = self.fc1(x)
        x = self.relu(x)
        # 全连接层2, 输出10个分类
        x = self.fc2(x)
        return x

# 定义评估函数
def evaluate_accuracy(data_iter, net, device):
    if not any(True for _ in data_iter):  # 检查data_iter是否为空
        return 0.0
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval()
            X = X.reshape((-1, 1, 24, 16)).to(device)
            y = y.to(device)
            y_hat = net(X)
            acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            net.train()
            n += y.shape[0]
    return acc_sum / n

# 定义训练函数
def train_cnn(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)  # 将模型移到CPU或GPU上，GPU加速计算
    print("training on", device)
    loss = nn.CrossEntropyLoss() 
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in data_iter(batch_size, data_train, tags_train):
            X = X.reshape((-1, 1, 24, 16)).to(device)  # 改变形状并移到device
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(data_iter(batch_size, data_test, tags_test), net, device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))


# 定义训练参数
lr, num_epochs = 0.02, 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 选择CPU或GPU
net = CNN()
optimizer = optim.Adam(net.parameters(), lr=lr)  # 相比于SGD, Adam优化器可以自动调整学习率
train_cnn(net, data_iter(batch_size, data_train, tags_train), data_iter(batch_size, data_test, tags_test), batch_size, optimizer, device, num_epochs)


training on cuda
epoch 1, loss 0.0787, train acc 0.106, test acc 0.130
epoch 2, loss 0.0721, train acc 0.158, test acc 0.130
epoch 3, loss 0.0714, train acc 0.170, test acc 0.315
epoch 4, loss 0.0429, train acc 0.566, test acc 0.963
epoch 5, loss 0.0094, train acc 0.953, test acc 0.981
epoch 6, loss 0.0076, train acc 0.974, test acc 0.944
epoch 7, loss 0.0048, train acc 0.977, test acc 0.963
epoch 8, loss 0.0055, train acc 0.974, test acc 0.963
epoch 9, loss 0.0034, train acc 0.974, test acc 0.963
epoch 10, loss 0.0026, train acc 0.977, test acc 0.963
epoch 11, loss 0.0033, train acc 0.974, test acc 0.963
epoch 12, loss 0.0023, train acc 0.982, test acc 0.963
epoch 13, loss 0.0027, train acc 0.979, test acc 0.944
epoch 14, loss 0.0022, train acc 0.977, test acc 0.944
epoch 15, loss 0.0029, train acc 0.971, test acc 0.963
epoch 16, loss 0.0033, train acc 0.979, test acc 0.944


### 训练模型保存

In [9]:
# 保存训练好的模型
torch.save(net.state_dict(), 'cnn_model.pth')

# 加载模型
net = CNN()
net.load_state_dict(torch.load('cnn_model.pth'))

<All keys matched successfully>